## Water Infrastructure and Agricultural Productivity in Morocco
Satellite Data Collection 

Research Question: How does water infrastructure modify the relationship between 
precipitation and agricultural productivity?

Study Area: Aoulouz Dam, Morocco (16km buffer)
Study Period: 1984-2010 (covering pre-dam, dam construction, and PPP implementation)
Key Infrastructure Events:
- 1991: Aoulouz Dam construction
- 1995: Public-Private Partnership (PPP) implementation  
- 2002: Second dam (Mokhtar Soussi) operational

Data Sources:
- Landsat 5 Collection 2 Surface Reflectance (NDVI, vegetation indices)
- CHIRPS precipitation (climate controls)

In [ ]:
import ee
import pandas as pd

In [ ]:
# Google Earth Engine initialization
ee.Authenticate()
ee.Initialize(project='ID') 

In [ ]:
# CONFIGURATION
# Study area coordinates (Aoulouz Dam, Morocco)
LON = -8.2727195
LAT = 30.639084
BUFFER_RADIUS_KM = 16

# Create area of interest (AOI)
point = ee.Geometry.Point([LON, LAT])
aoi = point.buffer(BUFFER_RADIUS_KM * 1000)  # Buffer in meters

# Temporal parameters
START_YEAR = 1984
END_YEAR = 2010
GROWING_SEASON_MONTHS = [5, 6, 7]  # May-July (critical irrigation period)

# Landsat 5 Surface Reflectance Collection 2
L5_COLLECTION = 'LANDSAT/LT05/C02/T1_L2'

print(f"Study area: {BUFFER_RADIUS_KM}km buffer around Aoulouz Dam ({LAT}, {LON})")
print(f"Study period: {START_YEAR}-{END_YEAR}")
print(f"Growing season months: {GROWING_SEASON_MONTHS}")

In [ ]:
# HELPER FUNCTIONS

def mask_landsat5_clouds(image):
    """
    Mask clouds, shadows, snow and water from Landsat 5 Collection 2 images.
    Apply scaling factors for Surface Reflectance bands.
    """
    qa_band = image.select('QA_PIXEL')
    
    # QA_PIXEL bit positions for Collection 2
    DILATED_CLOUD_BIT = 1
    CLOUD_BIT = 3
    CLOUD_SHADOW_BIT = 4
    SNOW_BIT = 5
    WATER_BIT = 7
    
    # Create masks for clear pixels (bit = 0 means clear)
    dilated_cloud_mask = qa_band.bitwiseAnd(1 << DILATED_CLOUD_BIT).eq(0)
    cloud_mask = qa_band.bitwiseAnd(1 << CLOUD_BIT).eq(0)
    cloud_shadow_mask = qa_band.bitwiseAnd(1 << CLOUD_SHADOW_BIT).eq(0)
    snow_mask = qa_band.bitwiseAnd(1 << SNOW_BIT).eq(0)
    water_mask = qa_band.bitwiseAnd(1 << WATER_BIT).eq(0)
    
    # Combine all masks
    combined_mask = (dilated_cloud_mask
                    .And(cloud_mask)
                    .And(cloud_shadow_mask)
                    .And(snow_mask)
                    .And(water_mask))
    
    # Apply scaling factors to Surface Reflectance bands
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    
    return image.addBands(optical_bands, None, True).updateMask(combined_mask)

def get_days_in_month(year, month):
    """Get number of days in a month, accounting for leap years."""
    if month in [1, 3, 5, 7, 8, 10, 12]:
        return 31
    elif month in [4, 6, 9, 11]:
        return 30
    else:  # February
        return 29 if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)) else 28

#### 1. LANDSAT 5 DATA COLLECTION - NDVI BANDS


In [ ]:
# Bands needed for NDVI: Red (SR_B3) and NIR (SR_B4)
NDVI_BANDS = ['SR_B3', 'SR_B4']
NDVI_DRIVE_FOLDER = 'GEE_Landsat5_NDVI_Morocco'

print(f"Bands: {NDVI_BANDS} (Red and NIR for NDVI)")
print(f"Export folder: {NDVI_DRIVE_FOLDER}")

# Collect NDVI bands for each year and month
for year in range(START_YEAR, END_YEAR + 1):
    for month in GROWING_SEASON_MONTHS:
        print(f"Processing {year}-{month:02d}")
        
        # Define date range
        start_date = f'{year}-{month:02d}-01'
        end_date = f'{year}-{month:02d}-{get_days_in_month(year, month):02d}'
        
        # Filter and process collection
        collection = (ee.ImageCollection(L5_COLLECTION)
                     .filterDate(start_date, end_date)
                     .filterBounds(aoi)
                     .map(mask_landsat5_clouds)
                     .select(NDVI_BANDS))
        
        try:
            collection_size = collection.size().getInfo()
            
            if collection_size > 0:
                print(f"  Found {collection_size} images")
                
                # Create monthly composite
                composite = collection.mean()
                
                # Export to Google Drive
                file_name = f'Aoulouz_L5_NDVI_{year}_{month:02d}'
                
                task = ee.batch.Export.image.toDrive(
                    image=composite.clip(aoi),
                    description=file_name,
                    folder=NDVI_DRIVE_FOLDER,
                    fileNamePrefix=file_name,
                    scale=30,
                    region=aoi.getInfo()['coordinates'],
                    maxPixels=1e10,
                    crs='EPSG:4326'
                )
                
                task.start()
                print(f"  ✓ Export started: {file_name}")
                
            else:
                print(f"  ✗ No images found for {year}-{month:02d}")
                
        except Exception as e:
            print(f"  ✗ Error processing {year}-{month:02d}: {e}")

#### 2. PRECIPITATION DATA COLLECTION - CHIRPS
* CHIRPS provides climate controls for causal identification
* Spatial resolution: ~5km (aggregated to AOI mean)
* Temporal coverage: Daily data aggregated to annual/monthly/seasonal


In [ ]:
# CHIRPS daily precipitation collection
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')

# 2a. Annual precipitation totals
print("\n2a. Processing annual precipitation...")
annual_precip_data = []

for year in range(START_YEAR, END_YEAR + 1):
    annual_sum = chirps.filterDate(f'{year}-01-01', f'{year}-12-31').sum()
    
    try:
        mean_precip = annual_sum.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=5000,  # CHIRPS native resolution
            maxPixels=1e10
        ).get('precipitation').getInfo()
        
        annual_precip_data.append([year, mean_precip])
        print(f"  {year}: {mean_precip:.2f} mm/year")
        
    except Exception as e:
        print(f"  Error processing {year}: {e}")
        annual_precip_data.append([year, None])

# Save annual precipitation
df_annual = pd.DataFrame(annual_precip_data, columns=['year', 'annual_precip_mm'])
df_annual.to_csv('chirps_annual_precipitation_morocco.csv', index=False)
print(f"✓ Annual precipitation saved to 'chirps_annual_precipitation_morocco.csv'")

# 2b. Growing season monthly precipitation
print("\n2b. Processing growing season monthly precipitation...")
monthly_precip_data = []

for year in range(START_YEAR, END_YEAR + 1):
    for month in GROWING_SEASON_MONTHS:
        start_date = f'{year}-{month:02d}-01'
        end_date = f'{year}-{month:02d}-{get_days_in_month(year, month):02d}'
        
        monthly_sum = chirps.filterDate(start_date, end_date).sum()
        
        try:
            mean_precip = monthly_sum.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=aoi,
                scale=5000,
                maxPixels=1e10
            ).get('precipitation').getInfo()
            
            monthly_precip_data.append([year, month, mean_precip])
            print(f"  {year}-{month:02d}: {mean_precip:.2f} mm")
            
        except Exception as e:
            print(f"  Error processing {year}-{month:02d}: {e}")
            monthly_precip_data.append([year, month, None])

# Save monthly precipitation
df_monthly = pd.DataFrame(monthly_precip_data, 
                         columns=['year', 'month', 'monthly_precip_mm'])
df_monthly.to_csv('chirps_monthly_precipitation_morocco.csv', index=False)
print(f"✓ Monthly precipitation saved to 'chirps_monthly_precipitation_morocco.csv'")

# 2c. Previous wet season precipitation (October year-1 to April year)
print("\n2c. Processing previous wet season precipitation...")
wet_season_data = []

for year in range(START_YEAR, END_YEAR + 1):
    # Wet season: October (year-1) to April (year)
    start_date = f'{year-1}-10-01'
    end_date = f'{year}-04-{get_days_in_month(year, 4):02d}'
    
    wet_season_sum = chirps.filterDate(start_date, end_date).sum()
    
    try:
        mean_precip = wet_season_sum.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=5000,
            maxPixels=1e10
        ).get('precipitation').getInfo()
        
        wet_season_data.append([year, mean_precip])
        print(f"  {year} (Oct {year-1} - Apr {year}): {mean_precip:.2f} mm")
        
    except Exception as e:
        print(f"  Error processing wet season {year}: {e}")
        wet_season_data.append([year, None])

# Save wet season precipitation
df_wet = pd.DataFrame(wet_season_data, 
                     columns=['year', 'wet_season_precip_mm'])
df_wet.to_csv('chirps_wet_season_precipitation_morocco.csv', index=False)
print(f"✓ Wet season precipitation saved to 'chirps_wet_season_precipitation_morocco.csv'")